In [1]:
import json
import pickle
import random
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet')
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [10]:
class ChatBot():
  def __init__(self, path):
    self.words = []
    self.classes = []
    self.documents = []
    ignore_words = ['?', '!']
    dataset = open(path).read()
    intents = json.loads(dataset)
    for intent in intents['intents']:
      for pattern in intent['patterns']:
        n_words = nltk.word_tokenize(pattern)
        self.words.extend(n_words)
        self.documents.append((n_words, intent['tag']))
        if intent['tag'] not in self.classes:
          self.classes.append(intent['tag'])
    temp = []
    for word in self.words:
      if word != "?" or word != ".":
        m_word = lemmatizer.lemmatize(word.lower())
        temp.append(m_word)
    self.words = sorted(list(set(temp)))
    self.classes = sorted(list(set(self.classes)))
    

  def create_model(self, path):
    self.model = Sequential()
    self.model.add(Dense(160, input_shape = (len(self.X[0]),), activation = "relu"))
    self.model.add(Dropout(0.5))
    self.model.add(Dense(80, activation = "relu"))
    self.model.add(Dropout(0.5))
    self.model.add(Dense(40, activation = "relu"))
    self.model.add(Dropout(0.5))
    self.model.add(Dense(len(self.y[0]), activation = "softmax"))
    sgd = SGD(learning_rate = 0.02, decay = 1e-5, momentum = 0.8, nesterov = True)
    self.model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])
    weights = self.model.fit(np.array(self.X), np.array(self.y), epochs = 150, batch_size = 5, verbose = 1)
    self.model.save(path, weights)
    

  def create_train_data(self):
    data = []
    zeros = [0] * len(self.classes)
    for document in self.documents:
      bool_vec = []
      patterns = document[0]
      patterns = [lemmatizer.lemmatize(pattern.lower()) for pattern in patterns]
      for w in self.words:
        if w in patterns:
          bool_vec.append(1)
        else:
          bool_vec.append(0)
      one_hot = zeros.copy()
      one_hot[self.classes.index(document[1])] = 1
      data.append([bool_vec, one_hot])
    random.shuffle(data)
    data = np.array(data)
    self.X = list(data[:,0])
    self.y = list(data[:,1])

In [ ]:
cb = ChatBot("./intents.json")
cb.create_train_data()
cb.create_model("./model.h5")